In [22]:
import pandas as pd
import numpy as np

In [23]:
# Upload files
commercials = pd.read_csv("../data/raw/commercials.csv")
comments = pd.read_csv("../data/raw/comments.csv")
sentiments = pd.read_csv("../data/processed/sentiment.csv")

In [24]:
# Check for shape in files
print("Shape of the commercials file: ", commercials.shape)
print("Shape of the comments file: ", comments.shape)
print("Shape of the sentiments file: ", comments.shape)

Shape of the commercials file:  (159, 8)
Shape of the comments file:  (4836, 6)
Shape of the sentiments file:  (4836, 6)


In [25]:
# Check for duplicate pairs in files
duplicate_commercials = commercials.duplicated(subset=['video_id', 'video_title', 'views', 'year']) 
print("Duplicate pairs in file commercial:", duplicate_commercials.sum())

duplicate_comments = comments.duplicated(subset=['video_id', 'video_title', 'link','author', 'text'])
print("Duplicate pairs in file comments:", duplicate_comments.sum())

duplicate_sentiments = sentiments.duplicated(subset=['video_id', 'video_title', 'link', 'author', 'text'])
print("Duplicate pairs in file comments:", duplicate_sentiments.sum())


Duplicate pairs in file commercial: 0
Duplicate pairs in file comments: 0
Duplicate pairs in file comments: 0


Make the necessary joins to create the final working df

In [26]:
# Join the commercial DataFrame of the commercial with the languaje detected in comments DataFrame
 
comments['video_id'] = comments['video_id'].astype(str)
commercials['video_id'] = commercials['video_id'].astype(str)

comments_language = comments.merge(commercials[['video_id', 'views', 'year', 'link', 'language_detected']],on=['video_id', 'link'],how='left')
comments_language.head(2)

,video_id,video_title,link,author,text,likes,views,year,language_detected
0,l9r673GbR8k,SHOW DE LUCES CON DRONES DE INTEL EN LA CARAV...,https://www.youtube.com/watch?v=l9r673GbR8k,@angiielozz9118,Fue genial,0,7853189,2016,en
1,l9r673GbR8k,SHOW DE LUCES CON DRONES DE INTEL EN LA CARAV...,https://www.youtube.com/watch?v=l9r673GbR8k,@EMILY-kp1cg,"...........""...,,,,',,,,,,,,,",0,7853189,2016,en


In [27]:
# Merge the sentiments DataFrame of the comment analysis with the results of the previous merge DataFrame
comments_language_sentiments = comments_language.merge( sentiments[['video_id', 'video_title', 'link', 'author', 'text', 'sentiment', 'confidence_score']], on=['video_id', 'video_title', 'link', 'author','text'], how='left')
comments_language_sentiments.head(1)

,video_id,video_title,link,author,text,likes,views,year,language_detected,sentiment,confidence_score
0,l9r673GbR8k,SHOW DE LUCES CON DRONES DE INTEL EN LA CARAV...,https://www.youtube.com/watch?v=l9r673GbR8k,@angiielozz9118,Fue genial,0,7853189,2016,en,neutral,0.645249


In [28]:
print("Shape of the comments_language: ", comments_language.shape)
print("Shape of the comments_language_sentiments: ", comments_language_sentiments.shape)

Shape of the comments_language:  (4836, 9)
Shape of the comments_language_sentiments:  (4836, 11)


In [29]:
duplicate_comments_language = comments_language.duplicated(subset=['video_id', 'video_title', 'link', 'author', 'text'])
print("Duplicate pairs in df comments_language:", duplicate_comments_language.sum())

duplicate_comments_language_sentiments = comments_language_sentiments.duplicated(subset=['video_id', 'video_title', 'link', 'author', 'text'])
print("Duplicate pairs in df comments_language_sentiments:", duplicate_comments_language_sentiments.sum())

Duplicate pairs in df comments_language: 0
Duplicate pairs in df comments_language_sentiments: 0


In [30]:
# The final DataFrame with comments, language, and sentiment analysis is assigned to the main variable 'df' for further use
df = comments_language_sentiments

In [31]:
# Counts how many video titles and unique IDs there are in the DataFrame, to analyze the diversity of the data.
df[['video_id', 'video_title']].nunique()
# Count how many unique video IDs (54) and titles (52) there are in the DataFrame.
# This suggests that some IDs may share the same title or have been duplicated.

video_id       54
video_title    52
dtype: int64

In [32]:
# Video titles associated with multiple video IDs (potential duplicates) are detected.
# Unique combinations of title, ID, link, and number of comments are then filtered and displayed to check for duplicates.

df['video_id'] =df['video_id'].astype(str)
duplicate = comments_language.groupby('video_title')['video_id'].nunique()
duplicate_titles = duplicate[duplicate > 1].index
df_duplicate =df[df['video_title'].isin(duplicate_titles)]
results = df_duplicate[['video_id', 'video_title', 'link', 'views', 'year', ]].drop_duplicates().sort_values('video_title')
results

,video_id,video_title,link,views,year
3991,Kt5rE0JnlUA,Coca-Cola® Obra de Arte,https://www.youtube.com/watch?v=Kt5rE0JnlUA,8706903,2023
4175,kXtupr6dlCI,Coca-Cola® Obra de Arte,https://www.youtube.com/watch?v=kXtupr6dlCI,5823569,2023
2045,47m0I4uU5mw,"Öyle ya da böyle, muhteşem Coca-Cola tadıyla y...",https://www.youtube.com/watch?v=47m0I4uU5mw,5478973,2018
2145,fnFKMxxZYTs,"Öyle ya da böyle, muhteşem Coca-Cola tadıyla y...",https://www.youtube.com/watch?v=fnFKMxxZYTs,5460168,2018


When filtering and displaying unique combinations, we see cases like:
- "Coca-Cola® Obra de Arte" and "Öyle ya da böyle..." which have different video_ids but share the same title,
- This could be due to re-releases, regional versions, or upload errors. <n>

To do this, each URL will be reviewed, as there are only four in this case, to manually determine if there is any discrepancy. Criteria for discarding the URLs will be determined later.

In [33]:
# List of video IDs that share a title with another video.
video_ids = [
    'Kt5rE0JnlUA',  # Coca-Cola® Obra de Arte
    'kXtupr6dlCI',  # Coca-Cola® Obra de Arte
    '47m0I4uU5mw',  # Öyle ya da böyle, muhteşem Coca-Cola tadıyla y...
    'fnFKMxxZYTs'   # Öyle ya da böyle, muhteşem Coca-Cola tadıyla y...
]
# This will be used to specifically analyze or handle these duplicate cases.

The yt_dlp library is used to obtain additional details (length and publication date) for duplicate videos without having to download them. This helps compare versions and decide which to keep or highlight.

In [34]:
import yt_dlp

ydl_opts = {
    'quiet': True,  # To avoid printing too much information
    'force_generic_extractor': True,  # Use a generic extractor
    'extract_flat': True,  # Only get metadata without downloading the video
}

# Function to get video duration and publish date
def get_video_details(video_id):
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        video_info = ydl.extract_info(f"https://www.youtube.com/watch?v={video_id}", download=False)
        duration = video_info.get('duration', 0)  # Duration in seconds
        publish_date = video_info.get('upload_date', 'N/A')  # Publish date (YYYYMMDD)
        
        # Convert duration to hours:minutes:seconds format
        duration_hms = f"{duration // 3600}:{(duration % 3600) // 60}:{duration % 60}"
        
        return {
            "video_id": video_id,
            "duration_hms": duration_hms,
            "publish_date": publish_date
        }
video_details = [get_video_details(video_id) for video_id in video_ids]

In [35]:
df_detals = pd.DataFrame(video_details)
df_detals

,video_id,duration_hms,publish_date
0,Kt5rE0JnlUA,0:0:30,20230307
1,kXtupr6dlCI,0:1:52,20230307
2,47m0I4uU5mw,0:0:21,20181023
3,fnFKMxxZYTs,0:0:47,20180927


Key details were obtained from the duplicate videos:
- They all share the same publication date or are very close in date. This could mean that one is the full version and the other is a short/advertised version released around the same time.
- The only differences observed are their length:
- For example: "Coca-Cola® Obra de Arte" has 30-second and 1:52-second versions.
- For example: "Öyle ya da böyle..." has 21-second and 47-second versions.

Therefore, we will continue to retain both videos, as they both meet the criteria established at the beginning of 5M+ views and 50+ comments. They will only be tagged "(Short Version)" <n> 

id: 'Kt5rE0JnlUA', '47m0I4uU5mw'.

In [36]:
# List of video IDs you want to modify the title for
ids_to_modify = ['Kt5rE0JnlUA', '47m0I4uU5mw']
df.loc[df['video_id'].isin(ids_to_modify), 'video_title'] += ' (Short Version)'

In [37]:
# The number of unique video IDs and titles in the DataFrame is counted again
df[['video_id', 'video_title']].nunique() 
# The results show that there are 54 titles and 54 unique IDs, indicating no additional duplicates in the titles

video_id       54
video_title    54
dtype: int64

In [61]:
df_final = df.rename(columns={
    'text': 'comment',
    'author': 'author_comment',
    'link': 'video_link',
    'views': 'video_views',
    'year': 'publish_video_year',
    'likes': 'likes_comment',
    'language_detected': 'video_title_language',
    'sentiment' : 'sentiment_comment'
})

# # Convert column data types
df_final = df_final.astype({
    'video_id': 'string',
    'video_title': 'string',
    'video_link': 'string',
    'author_comment': 'string',
    'comment': 'string',
    'likes_comment': 'int',
    'video_views': 'int',
    'publish_video_year': 'int',
    'video_title_language': 'string',
    'sentiment_comment': 'string',
    'confidence_score': 'float'
})

# Sort Columns
df_final = df_final[
    [
        'video_id',
        'video_title',
        'video_link',
        'video_views',
        'publish_video_year',
        'video_title_language',
        'author_comment',
        'comment',
        'likes_comment',
        'sentiment_comment',
        'confidence_score'
    ]
]

We adjust everything to load the final file.

In [62]:
import os

output_path = "../data/processed/coca_cola.csv"

# Delete if exists
if os.path.isfile(output_path):
    os.remove(output_path)

df_final.to_csv(output_path, index=False, encoding='utf-8-sig')